<a href="https://colab.research.google.com/github/yee030/nsmc-huggingface/blob/main/nsmc_huggingface_koelectra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install matplotlib

In [2]:
# HuggingFace transformers 설치 및 NSMC 데이터셋 다운로드
!pip install transformers
!wget https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt
!wget https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 48.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 54.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 34.6 MB/s eta 0:00:00
--2023-10-22 17:19:27--  https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4893335 (4.7M) [text/plain]
Saving to: ‘ratings_test.txt’

ratings_test.txt    100%[===================>]   4.67M  --.-KB/s    in 0.05s   

2023-10-22 17:19:27 (87.7 MB/s) - ‘ratings_test.txt’ saved [4893335/4893335]

--2023-

In [3]:
!head ratings_train.txt
!head ratings_test.txt

id	document	label
9976970	아 더빙.. 진짜 짜증나네요 목소리	0
3819312	흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나	1
10265843	너무재밓었다그래서보는것을추천한다	0
9045019	교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정	0
6483659	사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다	1
5403919	막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.	0
7797314	원작의 긴장감을 제대로 살려내지못했다.	0
9443947	별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단 낫겟다 납치.감금만반복반복..이드라마는 가족도없다 연기못하는사람만모엿네	0
7156791	액션이 없는데도 재미 있는 몇안되는 영화	1
id	document	label
6270596	굳 ㅋ	1
9274899	GDNTOPCLASSINTHECLUB	0
8544678	뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아	0
6825595	지루하지는 않은데 완전 막장임... 돈주고 보기에는....	0
6723715	3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??	0
7898805	음악이 주가 된, 최고의 음악영화	1
6315043	진정한 쓰레기	0
6097171	마치 미국애니에서 튀어나온듯한 창의력없는 로봇디자인부터가,고개를 젖게한다	0
8932678	갈수록 개판되가는 중국영화 유치하고 내용없음 폼잡다 끝남 말도안되는 무기에 유치한cg남무 아 그립다 동사서독같은 영화가 이건 3류아류작이다	0


In [4]:
import pandas as pd
import torch
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, ElectraForSequenceClassification, AdamW
from tqdm.notebook import tqdm

In [5]:
# GPU 사용
device = torch.device("cuda")

In [6]:
class NSMCDataset(Dataset):

  def __init__(self, csv_file):
    # 데이터를 불러옵니다. 일부 값에 NaN(결측값)이 있을 수 있으므로 제거합니다.
    self.dataset = pd.read_csv(csv_file, sep='\t').dropna(axis=0)
    # 중복된 데이터를 제거합니다. 'document' 열을 기준으로 중복을 확인합니다.
    self.dataset.drop_duplicates(subset=['document'], inplace=True)
    # Hugging Face의 Transformers 라이브러리에서 한국어 전용 Electra 모델의 토크나이저를 사용합니다.
    self.tokenizer = AutoTokenizer.from_pretrained("monologg/koelectra-small-v2-discriminator")

    # 데이터셋의 간단한 통계 정보를 출력합니다.
    print(self.dataset.describe())

  def __len__(self):
    # 데이터셋의 총 길이를 반환합니다.
    return len(self.dataset)

  def __getitem__(self, idx):
    # 데이터셋에서 특정 인덱스의 행을 가져옵니다.
    row = self.dataset.iloc[idx, 1:3].values
    text = row[0]  # 텍스트 데이터
    y = row[1]     # 레이블 데이터 (감정 분류 등)

    # 토크나이저를 사용하여 입력 데이터를 토큰화하고 텐서 형태로 변환합니다.
    inputs = self.tokenizer(
        text,
        return_tensors='pt',    # PyTorch 텐서 형태로 반환합니다.
        truncation=True,        # 입력 텍스트가 최대 길이를 초과하면 잘라냅니다.
        max_length=256,         # 입력 텍스트의 최대 길이
        pad_to_max_length=True,  # 최대 길이에 맞춰 패딩을 추가합니다.
        add_special_tokens=True  # 특수 토큰을 추가합니다. (예: [CLS], [SEP])
    )

    input_ids = inputs['input_ids'][0]         # 토큰화된 입력 텍스트
    attention_mask = inputs['attention_mask'][0]  # 어텐션 마스크 (패딩 토큰에 대한 어텐션을 막기 위해 사용)

    # 토큰화된 입력 텍스트, 어텐션 마스크, 레이블을 반환합니다.
    return input_ids, attention_mask, y


In [7]:
train_dataset = NSMCDataset("ratings_train.txt")
test_dataset = NSMCDataset("ratings_test.txt")

                 id          label
count  1.461820e+05  146182.000000
mean   6.779186e+06       0.498283
std    2.919223e+06       0.499999
min    3.300000e+01       0.000000
25%    4.814832e+06       0.000000
50%    7.581160e+06       0.000000
75%    9.274760e+06       1.000000
max    1.027815e+07       1.000000
                 id         label
count  4.915700e+04  49157.000000
mean   6.752945e+06      0.502695
std    2.937158e+06      0.499998
min    6.010000e+02      0.000000
25%    4.777143e+06      0.000000
50%    7.565415e+06      1.000000
75%    9.260204e+06      1.000000
max    1.027809e+07      1.000000


In [8]:
model = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-base-v3-discriminator").to(device)

# 한번 실행해보기
# text, attention_mask, y = train_dataset[0]
# model(text.unsqueeze(0).to(device), attention_mask=attention_mask.unsqueeze(0).to(device))

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
# 모델 레이어 보기
model

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(35000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0-11): 12 x ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): L

In [10]:
epochs = 1
batch_size = 16

In [11]:
# AdamW 옵티마이저를 생성합니다. 모델의 파라미터와 학습률을 설정합니다.
optimizer = AdamW(model.parameters(), lr=5e-6)

# 학습 데이터를 미니배치로 분할하는 데이터로더를 생성합니다.
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# 테스트 데이터를 미니배치로 분할하는 데이터로더를 생성합니다.
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=True)


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
# 손실과 정확도를 저장할 리스트를 초기화합니다.
losses = []
accuracies = []

# 학습 에폭(epochs) 동안 반복합니다.
for i in range(epochs):
  total_loss = 0.0  # 현재 에폭의 총 손실 초기화
  correct = 0  # 현재 에폭의 정확한 예측 수 초기화
  total = 0  # 현재 에폭의 전체 데이터 수 초기화
  batches = 0  # 현재 에폭에서 처리한 미니배치 수 초기화

  # 모델을 학습 모드로 설정합니다.
  model.train()

  # train_loader에서 미니배치를 가져와 학습을 수행합니다.
  for input_ids_batch, attention_masks_batch, y_batch in tqdm(train_loader):
    optimizer.zero_grad()  # 그래디언트 초기화
    y_batch = y_batch.to(device)  # 레이블을 GPU로 이동
    y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]  # 모델에 입력을 전달하여 예측 수행
    loss = F.cross_entropy(y_pred, y_batch)  # 크로스 엔트로피 손실 계산
    loss.backward()  # 그래디언트 역전파
    optimizer.step()  # 옵티마이저로 파라미터 업데이트

    total_loss += loss.item()  # 현재 미니배치의 손실을 더해 총 손실 계산

    # 예측된 클래스와 실제 클래스를 비교하여 정확도 계산
    _, predicted = torch.max(y_pred, 1)
    correct += (predicted == y_batch).sum()
    total += len(y_batch)

    batches += 1
    if batches % 100 == 0:
      print("Batch Loss:", total_loss, "Accuracy:", correct.float() / total)

  # 현재 에폭의 손실과 정확도를 리스트에 추가
  losses.append(total_loss)
















  accuracies.append(correct.float() / total)
  print("Train Loss:", total_loss, "Accuracy:", correct.float() / total)


  0%|          | 0/9137 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2606: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Batch Loss: 69.23714363574982 Accuracy: tensor(0.5050, device='cuda:0')
Batch Loss: 138.24038273096085 Accuracy: tensor(0.5197, device='cuda:0')
Batch Loss: 206.48825973272324 Accuracy: tensor(0.5344, device='cuda:0')
Batch Loss: 273.15697741508484 Accuracy: tensor(0.5544, device='cuda:0')
Batch Loss: 339.9271570444107 Accuracy: tensor(0.5625, device='cuda:0')
Batch Loss: 403.7667950093746 Accuracy: tensor(0.5768, device='cuda:0')
Batch Loss: 465.4919438660145 Accuracy: tensor(0.5900, device='cuda:0')
Batch Loss: 526.4008750915527 Accuracy: tensor(0.6002, device='cuda:0')
Batch Loss: 584.8454048633575 Accuracy: tensor(0.6108, device='cuda:0')
Batch Loss: 643.9113036692142 Accuracy: tensor(0.6185, device='cuda:0')
Batch Loss: 703.4185432195663 Accuracy: tensor(0.6241, device='cuda:0')
Batch Loss: 760.8172011971474 Accuracy: tensor(0.6309, device='cuda:0')
Batch Loss: 820.0711757838726 Accuracy: tensor(0.6354, device='cuda:0')


In [ ]:
import matplotlib.pyplot as plt

# 에폭별 손실 그래프 그리기
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(losses.cpu(), marker='o', linestyle='-')  # .cpu()를 사용하여 GPU에서 CPU로 이동
plt.title('Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.grid(True)

# 에폭별 정확도 그래프 그리기
plt.subplot(1, 2, 2)
plt.plot(accuracies.cpu(), marker='o', linestyle='-')  # .cpu()를 사용하여 GPU에서 CPU로 이동
plt.title('Training Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.grid(True)

# 그래프를 보여줍니다.
plt.tight_layout()
plt.show()


In [ ]:
# 모델을 평가 모드로 설정합니다. 이 모드에서는 드롭아웃과 배치 정규화와 같은 학습 관련 기능이 비활성화됩니다.
model.eval()

# 테스트 데이터셋에서 정확도를 계산하기 위한 초기 변수 설정
test_correct = 0  # 정확하게 예측한 샘플 수
test_total = 0    # 총 샘플 수

# 테스트 데이터셋에 대해 예측과 정확도를 계산합니다.
for input_ids_batch, attention_masks_batch, y_batch in tqdm(test_loader):
  y_batch = y_batch.to(device)  # 레이블을 GPU로 이동
  y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]  # 모델에 입력을 전달하여 예측을 얻습니다.
  _, predicted = torch.max(y_pred, 1)  # 가장 높은 확률을 가진 클래스를 선택하여 예측합니다.
  test_correct += (predicted == y_batch).sum()  # 정확하게 예측한 수를 누적합니다.
  test_total += len(y_batch)  # 전체 샘플 수를 누적합니다.

# 정확도를 출력합니다. test_correct는 정확하게 예측한 샘플 수, test_total은 전체 샘플 수입니다.
print("Accuracy:", test_correct.float() / test_total)

In [ ]:
# 모델 저장하기
torch.save(model.state_dict(), "model.pt")